<a href="https://polly.elucidata.io/manage/workspaces?action=open_polly_notebook&amp;source=github&amp;path=ElucidataInc%2Fpolly-python%2Fblob%2Fmain%2FEnrich%2Fcohort_creation_demo.ipynb&amp;kernel=elucidata%2FGenomics-hail&amp;machine=medium" target="_parent"><img alt="Open in Polly" src="https://elucidatainc.github.io/PublicAssets/open_polly.svg"/></a>


# Welcome to Notebook demonstrating cohort journey on polly-python

The demo of this can be found in this video: https://www.loom.com/share/71c97f0cfd554a9fb2aa6256664c08a0

In [1]:
# please do not modify
from IPython.display import display_html
def restartkernel() :
    display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)

In [2]:
!sudo pip3 install polly-python --upgrade

Looking in indexes: https://pypi.org/simple, http://54.245.179.143:80/
     |████████████████████████████████| 1.5 MB 8.8 MB/s eta 0:00:01
     |████████████████████████████████| 149 kB 127.0 MB/s eta 0:00:01
     |████████████████████████████████| 52 kB 63.1 MB/s  eta 0:00:01
     |████████████████████████████████| 50 kB 91.1 MB/s eta 0:00:01
     |████████████████████████████████| 306 kB 117.8 MB/s eta 0:00:01
     |████████████████████████████████| 96 kB 116.8 MB/s eta 0:00:01
Using legacy 'setup.py install' for Cerberus, since package 'wheel' is not installed.
Using legacy 'setup.py install' for retrying, since package 'wheel' is not installed.
  Attempting uninstall: certifi
    Found existing installation: certifi 2021.5.30
    Uninstalling certifi-2021.5.30:
      Successfully uninstalled certifi-2021.5.30
    Running setup.py install for retrying ... done
  Attempting uninstall: Cerberus
    Found existing installation: Cerberus 1.3.4
    Uninstalling Cerberus-1.3.4:
      Succ

In [ ]:
restartkernel() #Pause for a few seconds before the kernel is refreshed

In [1]:
import os
from polly.auth import Polly
from polly.omixatlas import OmixAtlas
from polly.cohort import Cohort
from polly.workspaces import Workspaces
import pandas as pd
import urllib.parse as urlparse
from urllib.parse import parse_qs
import numpy as np
import cmapPy

In [2]:
AUTH_TOKEN=(os.environ['POLLY_REFRESH_TOKEN']) # Obtain authentication tokens
Polly.auth(AUTH_TOKEN)
omixatlas = OmixAtlas()
cohort1 = Cohort()
cohort2 = Cohort()
workspaces = Workspaces()

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('display.width', 500)

## Finding datasets to be added in the cohort

In [3]:
query = """SELECT src_dataset_id, disease, protein_position, amino_acids, sequencer, impact, variant_class, consequence, name
        FROM tcga.features AS features
        
        JOIN (
        SELECT dataset_id AS dataset_id, curated_disease AS disease FROM tcga.datasets WHERE data_type LIKE 'Mutation'
         ) AS datasets
         ON features.src_dataset_id = datasets.dataset_id
         
         WHERE hugo_symbol IN ('TP53','PIK3CA','CDH1','GATA3') AND features.src_dataset_id LIKE '%BRCA%'
    ORDER BY features.src_dataset_id"""
results=omixatlas.query_metadata(query, query_api_version="v2")
results

Query execution succeeded (time taken: 32.27 seconds, data scanned: 12.670 MB)
Fetched 652 rows


,src_dataset_id,disease,protein_position,amino_acids,sequencer,impact,variant_class,consequence,name
0,BRCA_Mutation_TCGA-3C-AALI-01A-11D-A41F-09,['Breast Neoplasms'],183/393,S/*,Illumina HiSeq 2000,HIGH,SNV,stop_gained,TP53_c.548C>A
1,BRCA_Mutation_TCGA-3C-AALK-01A-11D-A41F-09,['Breast Neoplasms'],542/1068,E/K,Illumina HiSeq 2000,MODERATE,SNV,missense_variant,PIK3CA_c.1624G>A
2,BRCA_Mutation_TCGA-3C-AALK-01A-11D-A41F-09,['Breast Neoplasms'],1004/1068,M/I,Illumina HiSeq 2000,MODERATE,SNV,missense_variant,PIK3CA_c.3012G>T
3,BRCA_Mutation_TCGA-4H-AAAK-01A-12D-A41F-09,['Breast Neoplasms'],571/882,NaN,Illumina HiSeq 2000,HIGH,SNV,splice_donor_variant,CDH1_c.1711+1G>T
4,BRCA_Mutation_TCGA-5L-AAT0-01A-12D-A41F-09,['Breast Neoplasms'],23/882,Q/*,Illumina HiSeq 2000,HIGH,SNV,stop_gained,CDH1_c.67C>T
...,...,...,...,...,...,...,...,...,...
647,BRCA_Mutation_TCGA-XX-A899-01A-11D-A36J-09,['Breast Neoplasms'],83/1068,F/S,Illumina HiSeq 2000,MODERATE,SNV,missense_variant,PIK3CA_c.248T>C
648,BRCA_Mutation_TCGA-XX-A899-01A-11D-A36J-09,['Breast Neoplasms'],511/882,Q/*,Illumina HiSeq 2000,HIGH,SNV,stop_gained,CDH1_c.1531C>T
649,BRCA_Mutation_TCGA-XX-A89A-01A-11D-A36J-09,['Breast Neoplasms'],542/1068,E/K,Illumina HiSeq 2000,MODERATE,SNV,missense_variant,PIK3CA_c.1624G>A
650,BRCA_Mutation_TCGA-XX-A89A-01A-11D-A36J-09,['Breast Neoplasms'],129/882,Q/*,Illumina HiSeq 2000,HIGH,SNV,stop_gained;splice_region_variant,CDH1_c.385C>T


### Creating list of datasets which are to be added in the cohort

In [4]:
moderate_data = results.loc[results['impact'] == 'MODERATE', 'src_dataset_id'].tolist()
high_data = results.loc[results['impact'] == 'HIGH', 'src_dataset_id'].tolist()

In [5]:
high_data

['BRCA_Mutation_TCGA-3C-AALI-01A-11D-A41F-09',
 'BRCA_Mutation_TCGA-4H-AAAK-01A-12D-A41F-09',
 'BRCA_Mutation_TCGA-5L-AAT0-01A-12D-A41F-09',
 'BRCA_Mutation_TCGA-A1-A0SP-01A-11D-A099-09',
 'BRCA_Mutation_TCGA-A2-A0CK-01A-11D-A228-09',
 'BRCA_Mutation_TCGA-A2-A0CW-01A-21D-A10Y-09',
 'BRCA_Mutation_TCGA-A2-A0D0-01A-11W-A019-09',
 'BRCA_Mutation_TCGA-A2-A0D1-01A-11W-A050-09',
 'BRCA_Mutation_TCGA-A2-A0EW-01A-21D-A10Y-09',
 'BRCA_Mutation_TCGA-A2-A0EX-01A-21W-A050-09',
 'BRCA_Mutation_TCGA-A2-A0YG-01A-21D-A10G-09',
 'BRCA_Mutation_TCGA-A2-A0YJ-01A-11D-A10G-09',
 'BRCA_Mutation_TCGA-A2-A1G1-01A-21D-A13L-09',
 'BRCA_Mutation_TCGA-A2-A3Y0-01A-11D-A23C-09',
 'BRCA_Mutation_TCGA-A2-A4S2-01A-12D-A25Q-09',
 'BRCA_Mutation_TCGA-A7-A13D-01A-13D-A272-09',
 'BRCA_Mutation_TCGA-A7-A4SE-01A-11D-A25Q-09',
 'BRCA_Mutation_TCGA-A7-A6VV-01A-22D-A33E-09',
 'BRCA_Mutation_TCGA-A8-A075-01A-11D-A099-09',
 'BRCA_Mutation_TCGA-A8-A07O-01A-11W-A019-09',
 'BRCA_Mutation_TCGA-A8-A09Q-01A-11W-A019-09',
 'BRCA_Mutati

In [14]:
moderate_data = np.unique(moderate_data).tolist()
moderate_data[0:20]

['BRCA_Mutation_TCGA-3C-AALK-01A-11D-A41F-09',
 'BRCA_Mutation_TCGA-5L-AAT0-01A-12D-A41F-09',
 'BRCA_Mutation_TCGA-5L-AAT1-01A-12D-A41F-09',
 'BRCA_Mutation_TCGA-A1-A0SE-01A-11D-A099-09',
 'BRCA_Mutation_TCGA-A1-A0SI-01A-11D-A142-09',
 'BRCA_Mutation_TCGA-A1-A0SK-01A-12D-A099-09',
 'BRCA_Mutation_TCGA-A1-A0SO-01A-22D-A099-09',
 'BRCA_Mutation_TCGA-A2-A04N-01A-11D-A10Y-09',
 'BRCA_Mutation_TCGA-A2-A04P-01A-31D-A128-09',
 'BRCA_Mutation_TCGA-A2-A04T-01A-21W-A050-09',
 'BRCA_Mutation_TCGA-A2-A04V-01A-21W-A050-09',
 'BRCA_Mutation_TCGA-A2-A04W-01A-31D-A10Y-09',
 'BRCA_Mutation_TCGA-A2-A0CL-01A-11D-A10Y-09',
 'BRCA_Mutation_TCGA-A2-A0CP-01A-11W-A050-09',
 'BRCA_Mutation_TCGA-A2-A0CS-01A-11D-A10Y-09',
 'BRCA_Mutation_TCGA-A2-A0CW-01A-21D-A10Y-09',
 'BRCA_Mutation_TCGA-A2-A0D1-01A-11W-A050-09',
 'BRCA_Mutation_TCGA-A2-A0D3-01A-11D-A10Y-09',
 'BRCA_Mutation_TCGA-A2-A0EM-01A-11W-A050-09',
 'BRCA_Mutation_TCGA-A2-A0EN-01A-13D-A099-09']

### Creating cohort 1 (high impact) - while passing argument of repo and dataset

In this example, user has entered both optional parameters: repo_key and list of dataset_ids

#### create_cohort(local_path: str, cohort_name: str, description: str, repo_key= str (Optional), dataset_id= list (Optional)) 

In [7]:
cohort1.create_cohort("/import","high_data","datasets where impact of mutation marked High","tcga",high_data)

INFO:root:'98' dataset/s added to Cohort!
INFO:root:Cohort Created !


### Creating cohort 2 (moderate impact) - without passing argument of repo and dataset

In this example, user has entered only the mandatory parameters: location, name and description of cohort

In [16]:
cohort2.create_cohort("/import","moderate_data","datasets where impact of mutation marked Moderate")

INFO:root:Cohort Created !


### Addition of samples in cohort 2

In this example, user has entered both mandatory parameters: repo_key and list of dataset_ids

#### add_to_cohort(repo_key= str, dataset_id= list) 

In [17]:
cohort2.add_to_cohort("tcga",moderate_data[0:20])

INFO:root:'20' dataset/s added to Cohort!


### Summarizing the contents of the cohort

Pass two arguments for fetching the details of the cohort as shown below

In [18]:
metadata, cohort_details = cohort1.summarize_cohort()

In [19]:
metadata

{'cohort_name': 'high_data',
 'number_of_samples': 98,
 'description': 'datasets where impact of mutation marked High'}

In [20]:
cohort_details

,source_omixatlas,datatype,dataset_id
0,tcga,Mutation,BRCA_Mutation_TCGA-E2-A570-01A-11D-A29N-09
1,tcga,Mutation,BRCA_Mutation_TCGA-LL-A73Y-01A-11D-A33E-09
2,tcga,Mutation,BRCA_Mutation_TCGA-LL-A9Q3-01A-11D-A41F-09
3,tcga,Mutation,BRCA_Mutation_TCGA-EW-A1J2-01A-21D-A13L-09
4,tcga,Mutation,BRCA_Mutation_TCGA-LD-A7W6-01A-81D-A351-09
5,tcga,Mutation,BRCA_Mutation_TCGA-A8-A09Q-01A-11W-A019-09
6,tcga,Mutation,BRCA_Mutation_TCGA-AR-A5QM-01A-11D-A27P-09
7,tcga,Mutation,BRCA_Mutation_TCGA-A2-A1G1-01A-21D-A13L-09
8,tcga,Mutation,BRCA_Mutation_TCGA-A2-A0CK-01A-11D-A228-09
9,tcga,Mutation,BRCA_Mutation_TCGA-AO-A0JB-01A-11W-A071-09


## Merging the dataset level metadata for Cohort 1 (high.pco)

In [21]:
cohort1.merge_data("dataset")

,dataset_id,disease,kw_disease_type,kw_disease_stage,kw_sample_type,gender_x,kw_vital_status,kw_molecular_subtype,kw_drug,publication,patient_id,tissue,organism,kw_data_type,description,kw_cell_line,kw_cell_type,dataset_source,curation_version,total_num_samples,kw_repo,kw_package,kw_key,kw_bucket,kw_filetype,kw_region,kw_location,kw_timestamp
0,BRCA_Mutation_TCGA-E2-A570-01A-11D-A29N-09,[Breast Neoplasms],None,None,Tumor,gender,vital_status.x,LumA,[none],https://www.cell.com/cancer-cell/fulltext/S153...,TCGA-E2-A570-01A,[breast],Homo sapiens,Mutation,Breast invasive carcinoma RNASeq data for TCGA...,[None],[None],TCGA,g3,1,tcga,TCGA_data_lake/data,TCGA_data_lake/data/BRCA/Mutation/BRCA_Mutatio...,discover-prod-datalake-v1,gct,us-west-2,https://discover-prod-datalake-v1.s3-us-west-2...,1642897819373
1,BRCA_Mutation_TCGA-LL-A73Y-01A-11D-A33E-09,[Breast Neoplasms],None,None,Tumor,gender,vital_status.x,Basal,[none],https://www.cell.com/cancer-cell/fulltext/S153...,TCGA-LL-A73Y-01A,[breast],Homo sapiens,Mutation,Breast invasive carcinoma RNASeq data for TCGA...,[None],[None],TCGA,g3,1,tcga,TCGA_data_lake/data,TCGA_data_lake/data/BRCA/Mutation/BRCA_Mutatio...,discover-prod-datalake-v1,gct,us-west-2,https://discover-prod-datalake-v1.s3-us-west-2...,1642897813847
2,BRCA_Mutation_TCGA-LL-A9Q3-01A-11D-A41F-09,[Breast Neoplasms],None,None,Tumor,gender,vital_status.x,LumA,[none],https://www.cell.com/cancer-cell/fulltext/S153...,TCGA-LL-A9Q3-01A,[breast],Homo sapiens,Mutation,Breast invasive carcinoma RNASeq data for TCGA...,[None],[None],TCGA,g3,1,tcga,TCGA_data_lake/data,TCGA_data_lake/data/BRCA/Mutation/BRCA_Mutatio...,discover-prod-datalake-v1,gct,us-west-2,https://discover-prod-datalake-v1.s3-us-west-2...,1642897796393
3,BRCA_Mutation_TCGA-EW-A1J2-01A-21D-A13L-09,[Breast Neoplasms],None,None,Tumor,gender,vital_status.x,LumA,[none],https://www.cell.com/cancer-cell/fulltext/S153...,TCGA-EW-A1J2-01A,[breast],Homo sapiens,Mutation,Breast invasive carcinoma RNASeq data for TCGA...,[None],[None],TCGA,g3,1,tcga,TCGA_data_lake/data,TCGA_data_lake/data/BRCA/Mutation/BRCA_Mutatio...,discover-prod-datalake-v1,gct,us-west-2,https://discover-prod-datalake-v1.s3-us-west-2...,1642897663894
4,BRCA_Mutation_TCGA-LD-A7W6-01A-81D-A351-09,[Breast Neoplasms],None,None,Tumor,gender,vital_status.x,LumA,[none],https://www.cell.com/cancer-cell/fulltext/S153...,TCGA-LD-A7W6-01A,[breast],Homo sapiens,Mutation,Breast invasive carcinoma RNASeq data for TCGA...,[None],[None],TCGA,g3,1,tcga,TCGA_data_lake/data,TCGA_data_lake/data/BRCA/Mutation/BRCA_Mutatio...,discover-prod-datalake-v1,gct,us-west-2,https://discover-prod-datalake-v1.s3-us-west-2...,1642897811300
5,BRCA_Mutation_TCGA-A8-A09Q-01A-11W-A019-09,[Breast Neoplasms],None,None,Tumor,gender,vital_status.x,LumB,[none],https://www.cell.com/cancer-cell/fulltext/S153...,TCGA-A8-A09Q-01A,[breast],Homo sapiens,Mutation,Breast invasive carcinoma RNASeq data for TCGA...,[None],[None],TCGA,g3,1,tcga,TCGA_data_lake/data,TCGA_data_lake/data/BRCA/Mutation/BRCA_Mutatio...,discover-prod-datalake-v1,gct,us-west-2,https://discover-prod-datalake-v1.s3-us-west-2...,1642897592063
6,BRCA_Mutation_TCGA-AR-A5QM-01A-11D-A27P-09,[Breast Neoplasms],None,None,Tumor,gender,vital_status.x,LumA,[none],https://www.cell.com/cancer-cell/fulltext/S153...,TCGA-AR-A5QM-01A,[breast],Homo sapiens,Mutation,Breast invasive carcinoma RNASeq data for TCGA...,[None],[None],TCGA,g3,1,tcga,TCGA_data_lake/data,TCGA_data_lake/data/BRCA/Mutation/BRCA_Mutatio...,discover-prod-datalake-v1,gct,us-west-2,https://discover-prod-datalake-v1.s3-us-west-2...,1642897690756
7,BRCA_Mutation_TCGA-A2-A1G1-01A-21D-A13L-09,[Breast Neoplasms],None,None,Tumor,gender,vital_status.x,Basal,[none],https://www.cell.com/cancer-cell/fulltext/S153...,TCGA-A2-A1G1-01A,[breast],Homo sapiens,Mutation,Breast invasive carcinoma RNASeq data for TCGA...,[None],[None],TCGA,g3,1,tcga,TCGA_data_lake/data,TCGA_data_lake/data/BRCA/Mutation/BRCA_Mutatio...,discover-prod-datalake-v1,gct,us-west

In [22]:
cohort1.merge_data("sample")

,dataset_id,patient,barcode,sample,shortLetterCode,definition,sample_submitter_id,sample_type_id,oct_embedded,sample_id,submitter_id,state,is_ffpe,sample_type,tissue_type,days_to_collection,initial_weight,pathology_report_uuid,synchronous_malignancy,ajcc_pathologic_stage,tumor_stage,days_to_diagnosis,last_known_disease_status,tissue_or_organ_of_origin,days_to_last_follow_up,primary_diagnosis,age_at_diagnosis,prior_malignancy,year_of_diagnosis,prior_treatment,ajcc_staging_system_edition,ajcc_pathologic_t,morphology,ajcc_pathologic_n,ajcc_pathologic_m,classification_of_tumor,diagnosis_id,icd_10_code,site_of_resection_or_biopsy,tumor_grade,progression_or_recurrence,alcohol_history,exposure_id,race,ethnicity,gender,vital_status.x,age_at_index,days_to_birth.x,year_of_birth,demographic_id,bcr_patient_barcode,primary_site,project_id,disease_type,name,releasable,released,paper_patient,paper_Tumor.Type,paper_Included_in_previous_marker_papers,paper_vital_status,paper_days_to_birth,paper_days_to_death,paper_days_to_last_followup,paper_age_at_initial_pathologic_diagnosis,paper_pathologic_stage,paper_Tumor_Grade,paper_BRCA_Pathology,paper_BRCA_Subtype_PAM50,paper_MSI_status,paper_HPV_Status,paper_tobacco_smoking_history,paper_CNV.Clusters,paper_Mutation.Clusters,paper_DNA.Methylation.Clusters,paper_mRNA.Clusters,paper_miRNA.Clusters,paper_lncRNA.Clusters,paper_Protein.Clusters,paper_PARADIGM.Clusters,paper_Pan.Gyn.Clusters,Tumor.Type,Included_in_previous_marker_papers,vital_status.y,days_to_birth.y,days_to_death.y,days_to_last_followup,age_at_initial_pathologic_diagnosis,pathologic_stage,Tumor_Grade,BRCA_Pathology,BRCA_Subtype_PAM50,MSI_status,HPV_Status,tobacco_smoking_history,CNV.Clusters,Mutation.Clusters,DNA.Methylation.Clusters,mRNA.Clusters,miRNA.Clusters,lncRNA.Clusters,Protein.Clusters,PARADIGM.Clusters,Pan.Gyn.Clusters,subtype,kw_curated_cell_line,kw_curated_cell_type,kw_curated_genetic_mod_type,kw_curated_modified_gene,kw_curated_tissue,kw_curated_drug,kw_curated_disease,curated_age_unit,curated_min_age,curated_max_age,days_to_death.x,year_of_death
cid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
TCGA-E2-A570-01A-11D-A29N-09,BRCA_Mutation_TCGA-E2-A570-01A-11D-A29N-09,TCGA-E2-A570,TCGA-E2-A570-01A-11R-A29R-07,TCGA-E2-A570-01A,TP,Primary solid Tumor,TCGA-E2-A570-01A,1,true,da3319a2-f906-495c-b0de-38cb8672e28f,TCGA-E2-A570,released,FALSE,Primary Tumor,Not Reported,416,80,803922CD-3D37-47DC-8BDC-EF8F4779685A,No,Stage IB,stage ib,0,not reported,Breast-- NOS,931.0,Infiltrating duct carcinoma-- NOS,17456.0,no,2011,No,7th,T1c,8500/3,N1mi,M0,not reported,9cc17651-1122-53bd-b005-1faa310e856f,C50.9,Breast-- NOS,not reported,not reported,Not Reported,4dc34d32-13b6-5b86-b932-daee52a9451d,black or african american,not hispanic or latino,female,Alive,47,-17456.0,1964,3498eea3-4732-5c0a-8b9e-09599478966e,TCGA-E2-A570-01A,Breast,TCGA-BRCA,"c(""Ductal and Lobular Neoplasms""-- ""Complex Ep...",Breast Invasive Carcinoma,TRUE,TRUE,TCGA-E2-A570,BRCA,NO,Alive,-17456.0,NaN,931.0,47,Stage_I,NaN,IDC,LumA,NaN,NaN,NaN,C1,C4,C1,C2,C2,NaN,NaN,C8,NaN,BRCA,NO,Alive,-17456.0,NaN,931.0,47,Stage_I,NaN,IDC,LumA,NaN,NaN,NaN,C1,C4,C1,C2,C2,NaN,NaN,C8,NaN,LumA,none,none,none,none,breast,none,Breast Neoplasms,Years,47.0,47.0,NaN,NaN
TCGA-LL-A73Y-01A-11D-A33E-09,BRCA_Mutation_TCGA-LL-A73Y-01A-11D-A33E-09,TCGA-LL-A73Y,TCGA-LL-A73Y-01A-11R-A33J-07,TCGA-LL-A73Y-01A,TP,Primary solid Tumor,TCGA-LL-A73Y-01A,1,false,4d088b92-ccb7-44fe-a582-f1d9034cd643,TCGA-LL-A73Y,released,FALSE,Primary Tumor,Not Reported,55,60,9A8F7726-D97C-44DE-B72F-6BF0FC2078F8,No,Stage IA,stage ia,0,not reported,Breast-- NOS,477.0,Infiltrating duct carcinoma-- NOS,24647.0,no,2013,No,7th,T1c,8500/3,N0,MX,not reported,8de52698-2719-56fc-b24a-04f2b2e576f3,C50.9,Breast-- NOS,not reported,not reported,Not Reported,27b7f389-bc03-523f-8969-859467858195,black or african american,not hispanic or latino,female,Alive,67,-24647.0,1946,4a6f97c8

In [23]:
cohort1.merge_data("feature")

rhd,dataset_id,Hugo_Symbol,Entrez_Gene_Id,Center,NCBI_Build,Chromosome,Start_Position,End_Position,Strand,Variant_Classification,Variant_Type,Reference_Allele,Tumor_Seq_Allele1,Tumor_Seq_Allele2,dbSNP_RS,dbSNP_Val_Status,Tumor_Sample_Barcode,Matched_Norm_Sample_Barcode,Match_Norm_Seq_Allele1,Match_Norm_Seq_Allele2,Tumor_Validation_Allele1,Tumor_Validation_Allele2,Match_Norm_Validation_Allele1,Match_Norm_Validation_Allele2,Verification_Status,Validation_Status,Mutation_Status,Sequencing_Phase,Sequence_Source,Validation_Method,Score,BAM_File,Sequencer,Tumor_Sample_UUID,Matched_Norm_Sample_UUID,HGVSc,HGVSp,HGVSp_Short,Transcript_ID,Exon_Number,t_depth,t_ref_count,t_alt_count,n_depth,n_ref_count,n_alt_count,Allele,Gene,Feature,Feature_type,One_Consequence,Consequence,cDNA_position,CDS_position,Protein_position,Amino_acids,Codons,Existing_variation,ALLELE_NUM,DISTANCE,TRANSCRIPT_STRAND,SYMBOL,SYMBOL_SOURCE,HGNC_ID,BIOTYPE,CANONICAL,CCDS,ENSP,SWISSPROT,TREMBL,UNIPARC,RefSeq,SIFT,PolyPhen,EXON,INTRON,DOMAINS,GMAF,AFR_MAF,AMR_MAF,ASN_MAF,EAS_MAF,EUR_MAF,SAS_MAF,AA_MAF,EA_MAF,CLIN_SIG,SOMATIC,PUBMED,MOTIF_NAME,MOTIF_POS,HIGH_INF_POS,MOTIF_SCORE_CHANGE,IMPACT,PICK,VARIANT_CLASS,TSL,HGVS_OFFSET,PHENO,MINIMISED,ExAC_AF,ExAC_AF_Adj,ExAC_AF_AFR,ExAC_AF_AMR,ExAC_AF_EAS,ExAC_AF_FIN,ExAC_AF_NFE,ExAC_AF_OTH,ExAC_AF_SAS,GENE_PHENO,FILTER,CONTEXT,src_vcf_id,tumor_bam_uuid,normal_bam_uuid,case_id,GDC_FILTER,COSMIC,MC3_Overlap,GDC_Validation_Status
rid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
SPEN_c.2170C>T,BRCA_Mutation_TCGA-E2-A570-01A-11D-A29N-09,SPEN,23013,WUGSC,GRCh38,chr1,15928410,15928410,+,Nonsense_Mutation,SNP,C,C,T,NaN,NaN,TCGA-E2-A570-01A-11D-A29N-09,TCGA-E2-A570-10A-01D-A29N-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Somatic,NaN,NaN,NaN,NaN,NaN,Illumina HiSeq 2000,0ebb7058-4311-40a6-ac47-6b5f0c38492f,714b82cd-b87d-4c9f-af14-386a442d1c74,c.2170C>T,p.Arg724Ter,p.R724*,ENST00000375759,11/15,24,19,5,37,NaN,NaN,T,ENSG00000065526,ENST00000375759,Transcript,stop_gained,stop_gained,2374/12232,2170/10995,724/3664,R/*,Cga/Tga,NaN,1,NaN,1,SPEN,HGNC,HGNC:17575,protein_coding,YES,CCDS164.1,ENSP00000364912,Q96T58,NaN,NaN,NM_015001.2,NaN,NaN,11/15,NaN,PROSITE_profiles:PS50323,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HIGH,1.0,SNV,1.0,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PASS,TTGAACGAAGT,6d794197-c206-4e67-a4ab-678c78db2a90,2e57cace-9aeb-49c5-a58e-5ea5ebaebff1,dcd10a2a-0272-4a82-b835-994c41f727b4,cdd8c046-fd97-482a-a11f-2217b321c4d1,NaN,COSM3802617,TRUE,Unknown
TMEM57_c.1251C>G,BRCA_Mutation_TCGA-E2-A570-01A-11D-A29N-09,TMEM57,55219,WUGSC,GRCh38,chr1,25484212,25484212,+,Silent,SNP,C,C,G,NaN,NaN,TCGA-E2-A570-01A-11D-A29N-09,TCGA-E2-A570-10A-01D-A29N-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Somatic,NaN,NaN,NaN,NaN,NaN,Illumina HiSeq 2000,0ebb7058-4311-40a6-ac47-6b5f0c38492f,714b82cd-b87d-4c9f-af14-386a442d1c74,c.1251C>G,p.=,p.T417T,ENST00000374343,7/11,18,11,7,27,NaN,NaN,G,ENSG00000204178,ENST00000374343,Transcript,synonymous_variant,synonymous_variant,1430/3917,1251/1995,417/664,T,acC/acG,NaN,1,NaN,1,TMEM57,HGNC,HGNC:25572,protein_coding,YES,CCDS30638.1,ENSP00000363463,Q8N5G2,NaN,UPI000004A074,NM_018202.5,NaN,NaN,7/11,NaN,Pfam_domain:PF09726;Superfamily_domains:SSF46579,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LOW,1.0,SNV,1.0,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PASS,AGCACCGAGCG,6d794197-c206-4e67-a4ab-678c78db2a90,2e57cace-9aeb-49c5-a58e-5ea5ebaebff1,dcd10a2a-0272-4a82-b835-994c41f727b4,cdd8c046-fd97-482a-a11f-2217b321c4d1,NaN,COSM3804782,TRUE,Unknown
SSBP3_c.*377T>G,BRCA_Mutation_TCGA-E2-A570-01A-11D-A29N-09,SSBP3,23648,WUGSC,GRCh38,chr1,54226754,54226754,+,3'UTR,SNP,A,A,C,rs548654307,NaN,TCGA-E2-A570-01A-11D-A29N-09,TCGA-E2-A570-10A-01D-A29N-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Somatic,NaN,NaN,NaN,NaN,NaN,Illumina HiSeq 2000,0ebb7058-4311-40a6-ac47-6b5f0c38492f,714b82cd-b87d-4c9f-af14-386a442d1c74,c.*377T>G,NaN,NaN,ENS

In [24]:
cohort1.merge_data("data_matrix")

cid,TCGA-3C-AALI-01A-11D-A41F-09,TCGA-4H-AAAK-01A-12D-A41F-09,TCGA-5L-AAT0-01A-12D-A41F-09,TCGA-A1-A0SP-01A-11D-A099-09,TCGA-A2-A0CK-01A-11D-A228-09,TCGA-A2-A0CW-01A-21D-A10Y-09,TCGA-A2-A0D0-01A-11W-A019-09,TCGA-A2-A0D1-01A-11W-A050-09,TCGA-A2-A0EW-01A-21D-A10Y-09,TCGA-A2-A0EX-01A-21W-A050-09,TCGA-A2-A0YG-01A-21D-A10G-09,TCGA-A2-A0YJ-01A-11D-A10G-09,TCGA-A2-A1G1-01A-21D-A13L-09,TCGA-A2-A3Y0-01A-11D-A23C-09,TCGA-A2-A4S2-01A-12D-A25Q-09,TCGA-A7-A13D-01A-13D-A272-09,TCGA-A7-A4SE-01A-11D-A25Q-09,TCGA-A7-A6VV-01A-22D-A33E-09,TCGA-A8-A075-01A-11D-A099-09,TCGA-A8-A07O-01A-11W-A019-09,TCGA-A8-A09Q-01A-11W-A019-09,TCGA-A8-A09W-01A-11W-A019-09,TCGA-A8-A0A6-01A-12W-A071-09,TCGA-A8-A0A9-01A-11W-A019-09,TCGA-AC-A4ZE-01A-11D-A41F-09,TCGA-AC-A6IV-01A-12D-A33E-09,TCGA-AC-A8OQ-01A-11D-A41F-09,TCGA-AC-A8OS-01A-12D-A41F-09,TCGA-AN-A03Y-01A-21W-A019-09,TCGA-AN-A0FL-01A-11W-A050-09,TCGA-AN-A0G0-01A-11W-A050-09,TCGA-AN-A0XN-01A-21D-A10G-09,TCGA-AN-A0XT-01A-11D-A10G-09,TCGA-AO-A03L-01A-41W-A071-09,TCGA-AO-A0JB-01A-11W-A071-09,TCGA-AO-A124-01A-11D-A10M-09,TCGA-AO-A128-01A-11D-A10M-09,TCGA-AR-A0TS-01A-11D-A10Y-09,TCGA-AR-A1AP-01A-11D-A12Q-09,TCGA-AR-A24X-01A-11D-A167-09,TCGA-AR-A254-01A-21D-A167-09,TCGA-AR-A2LE-01A-11D-A17W-09,TCGA-AR-A5QM-01A-11D-A27P-09,TCGA-B6-A0IE-01A-11W-A050-09,TCGA-B6-A0RU-01A-11D-A099-09,TCGA-B6-A0X1-01A-11D-A10G-09,TCGA-B6-A2IU-01A-32D-A19T-09,TCGA-B6-A40B-01A-11D-A23C-09,TCGA-B6-A40C-01A-11D-A23C-09,TCGA-BH-A0B7-01A-12D-A10Y-09,TCGA-BH-A0BQ-01A-21D-A10Y-09,TCGA-BH-A0C0-01A-21W-A071-09,TCGA-BH-A0HP-01A-12D-A099-09,TCGA-BH-A18H-01A-11D-A12B-09,TCGA-BH-A18V-01A-11D-A12B-09,TCGA-BH-A1FC-01A-11D-A13L-09,TCGA-BH-A202-01A-11D-A14K-09,TCGA-BH-A208-01A-11D-A159-09,TCGA-BH-A209-01A-11D-A17G-09,TCGA-C8-A1HJ-01A-11D-A13L-09,TCGA-C8-A1HK-01A-21D-A13L-09,TCGA-D8-A143-01A-11D-A10Y-09,TCGA-D8-A1JK-01A-11D-A13L-09,TCGA-D8-A1XA-01A-11D-A14G-09,TCGA-D8-A1XO-01A-11D-A14K-09,TCGA-D8-A1XZ-01A-11D-A14K-09,TCGA-D8-A1Y1-01A-21D-A14K-09,TCGA-D8-A1Y3-01A-11D-A159-09,TCGA-D8-A27M-01A-11D-A16D-09,TCGA-D8-A27V-01A-12D-A17D-09,TCGA-E2-A14N-01A-31D-A135-09,TCGA-E2-A14X-01A-11D-A10Y-09,TCGA-E2-A150-01A-11D-A12B-09,TCGA-E2-A155-01A-11D-A12B-09,TCGA-E2-A15D-01A-11D-A10Y-09,TCGA-E2-A1II-01A-11D-A142-09,TCGA-E2-A1L8-01A-11D-A13L-09,TCGA-E2-A570-01A-11D-A29N-09,TCGA-E9-A1N9-01A-11D-A14G-09,TCGA-E9-A243-01A-21D-A167-09,TCGA-EW-A1J2-01A-21D-A13L-09,TCGA-EW-A1OY-01A-11D-A142-09,TCGA-EW-A1OZ-01A-11D-A142-09,TCGA-EW-A1PA-01A-11D-A142-09,TCGA-GM-A2DB-01A-31D-A19Y-09,TCGA-GM-A2DF-01A-11D-A17W-09,TCGA-GM-A2DH-01A-11D-A17W-09,TCGA-GM-A5PV-01A-11D-A28B-09,TCGA-LD-A66U-01A-11D-A31U-09,TCGA-LD-A7W6-01A-81D-A351-09,TCGA-LL-A50Y-01A-11D-A25Q-09,TCGA-LL-A73Y-01A-11D-A33E-09,TCGA-LL-A9Q3-01A-11D-A41F-09,TCGA-OL-A5RV-01A-12D-A28B-09,TCGA-OL-A5RX-01A-11D-A28B-09,TCGA-PE-A5DD-01A-12D-A27P-09,TCGA-XX-A899-01A-11D-A36J-09,TCGA-XX-A89A-01A-11D-A36J-09
A1CF_c.584C>T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A2ML1_c.1907C>G,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A2M_c.2257G>A,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Na

### Remove samples from cohort

In [25]:
cohort2.remove_from_cohort(moderate_data[0:10])

INFO:root:'10' dataset/s removed from Cohort!


### Checking the validity of the cohort

In [26]:
cohort2.is_valid()

True

### Summarize cohort to verify if the samples were removed 

In [27]:
metadata, cohort_details = cohort2.summarize_cohort()

In [28]:
metadata

{'cohort_name': 'moderate_data',
 'number_of_samples': 10,
 'description': 'datasets where impact of mutation marked Moderate'}

In [29]:
cohort_details

,source_omixatlas,datatype,dataset_id
0,tcga,Mutation,BRCA_Mutation_TCGA-A2-A0D1-01A-11W-A050-09
1,tcga,Mutation,BRCA_Mutation_TCGA-A2-A04V-01A-21W-A050-09
2,tcga,Mutation,BRCA_Mutation_TCGA-A2-A0EM-01A-11W-A050-09
3,tcga,Mutation,BRCA_Mutation_TCGA-A2-A0CW-01A-21D-A10Y-09
4,tcga,Mutation,BRCA_Mutation_TCGA-A2-A04W-01A-31D-A10Y-09
5,tcga,Mutation,BRCA_Mutation_TCGA-A2-A0CL-01A-11D-A10Y-09
6,tcga,Mutation,BRCA_Mutation_TCGA-A2-A0CS-01A-11D-A10Y-09
7,tcga,Mutation,BRCA_Mutation_TCGA-A2-A0D3-01A-11D-A10Y-09
8,tcga,Mutation,BRCA_Mutation_TCGA-A2-A0EN-01A-13D-A099-09
9,tcga,Mutation,BRCA_Mutation_TCGA-A2-A0CP-01A-11W-A050-09


## Editing the metadata of a cohort
Edit the name of the cohort and the description using this function

#### cohort.edit_cohort(name, description)

In [30]:
cohort2.edit_cohort("moderate_renamed","tcga_tp53_brca_mutation_moderate_impact")

INFO:root:Cohort Name Updated!
INFO:root:Cohort Description Updated!


### Summarize confirm to confirm if the cohort has been edited 

In [31]:
metadata, cohort_details = cohort2.summarize_cohort()

In [32]:
metadata

{'cohort_name': 'moderate_renamed',
 'number_of_samples': 10,
 'description': 'tcga_tp53_brca_mutation_moderate_impact'}

In [33]:
cohort_details

,source_omixatlas,datatype,dataset_id
0,tcga,Mutation,BRCA_Mutation_TCGA-A2-A0D1-01A-11W-A050-09
1,tcga,Mutation,BRCA_Mutation_TCGA-A2-A04V-01A-21W-A050-09
2,tcga,Mutation,BRCA_Mutation_TCGA-A2-A0EM-01A-11W-A050-09
3,tcga,Mutation,BRCA_Mutation_TCGA-A2-A0CW-01A-21D-A10Y-09
4,tcga,Mutation,BRCA_Mutation_TCGA-A2-A04W-01A-31D-A10Y-09
5,tcga,Mutation,BRCA_Mutation_TCGA-A2-A0CL-01A-11D-A10Y-09
6,tcga,Mutation,BRCA_Mutation_TCGA-A2-A0CS-01A-11D-A10Y-09
7,tcga,Mutation,BRCA_Mutation_TCGA-A2-A0D3-01A-11D-A10Y-09
8,tcga,Mutation,BRCA_Mutation_TCGA-A2-A0EN-01A-13D-A099-09
9,tcga,Mutation,BRCA_Mutation_TCGA-A2-A0CP-01A-11W-A050-09


## Delete the cohort

A cohort can be deleted by using delete_cohort() function

#### cohort.delete_cohort()

In [34]:
cohort2.delete_cohort()

INFO:root:Cohort Deleted Successfuly!


### Save cohort to the workspace

After editing the cohort, the user wants to save it in workspace for future use so they can use the following function

In [101]:
#Note: Preview of jpco and gct files are not available on worspace yet.

In [35]:
workspaces.upload_to_workspaces(workspace_id = 8861, workspace_path = "/high_brca_impact_p53.pco", 
                                local_path = "/import/high_data.pco")

INFO:root:Upload successful on workspace-id=8861.


### Upload cohort from the workspace

In [36]:
workspaces.download_from_workspaces(workspace_id = 8861, workspace_path = "/high_brca_impact_p53.pco")

INFO:root:Download successful to path=/import/high_brca_impact_p53.pco


### Loading the cohort which was uploaded from the workspace

In [37]:
cohort3 = Cohort()
cohort3.load_cohort("/import/high_brca_impact_p53.pco")

INFO:root:Cohort Loaded !


### Checking valididty of the cohort which we just uploaded from workspace

In [38]:
cohort3.is_valid()

True

### Checking the summary of the cohort

In [39]:
metadata, cohort_details = cohort3.summarize_cohort()

In [40]:
metadata

{'cohort_name': 'high_brca_impact_p53',
 'number_of_samples': 98,
 'description': 'datasets where impact of mutation marked High'}

In [41]:
cohort_details

,source_omixatlas,datatype,dataset_id
0,tcga,Mutation,BRCA_Mutation_TCGA-E2-A570-01A-11D-A29N-09
1,tcga,Mutation,BRCA_Mutation_TCGA-LL-A73Y-01A-11D-A33E-09
2,tcga,Mutation,BRCA_Mutation_TCGA-LL-A9Q3-01A-11D-A41F-09
3,tcga,Mutation,BRCA_Mutation_TCGA-EW-A1J2-01A-21D-A13L-09
4,tcga,Mutation,BRCA_Mutation_TCGA-LD-A7W6-01A-81D-A351-09
5,tcga,Mutation,BRCA_Mutation_TCGA-A8-A09Q-01A-11W-A019-09
6,tcga,Mutation,BRCA_Mutation_TCGA-AR-A5QM-01A-11D-A27P-09
7,tcga,Mutation,BRCA_Mutation_TCGA-A2-A1G1-01A-21D-A13L-09
8,tcga,Mutation,BRCA_Mutation_TCGA-A2-A0CK-01A-11D-A228-09
9,tcga,Mutation,BRCA_Mutation_TCGA-AO-A0JB-01A-11W-A071-09


### Adding files to cohort which was uploaded from workspace

In [45]:
cohort3.add_to_cohort("tcga",moderate_data[0:5])

INFO:root:The entity_id - BRCA_Mutation_TCGA-5L-AAT0-01A-12D-A41F-09 is already existing in the cohort.
INFO:root:'4' dataset/s added to Cohort!


### Verifying the files are added

In [46]:
metadata, cohort_details = cohort3.summarize_cohort()

In [47]:
metadata

{'cohort_name': 'high_brca_impact_p53',
 'number_of_samples': 102,
 'description': 'datasets where impact of mutation marked High'}

In [48]:
cohort_details

,source_omixatlas,datatype,dataset_id
0,tcga,Mutation,BRCA_Mutation_TCGA-E2-A570-01A-11D-A29N-09
1,tcga,Mutation,BRCA_Mutation_TCGA-LL-A73Y-01A-11D-A33E-09
2,tcga,Mutation,BRCA_Mutation_TCGA-LL-A9Q3-01A-11D-A41F-09
3,tcga,Mutation,BRCA_Mutation_TCGA-EW-A1J2-01A-21D-A13L-09
4,tcga,Mutation,BRCA_Mutation_TCGA-LD-A7W6-01A-81D-A351-09
5,tcga,Mutation,BRCA_Mutation_TCGA-A8-A09Q-01A-11W-A019-09
6,tcga,Mutation,BRCA_Mutation_TCGA-AR-A5QM-01A-11D-A27P-09
7,tcga,Mutation,BRCA_Mutation_TCGA-A2-A1G1-01A-21D-A13L-09
8,tcga,Mutation,BRCA_Mutation_TCGA-A2-A0CK-01A-11D-A228-09
9,tcga,Mutation,BRCA_Mutation_TCGA-AO-A0JB-01A-11W-A071-09
